In [1]:
#!pip install hypernetx

In [2]:
import hypernetx as hnx
import max_triplet as hnxmt

/opt/miniconda3/envs/hnx/lib/python3.11/site-packages/nptyping/typing_.py:51: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  Bool8 = np.bool8
/opt/miniconda3/envs/hnx/lib/python3.11/site-packages/nptyping/typing_.py:54: DeprecationWarning: `np.object0` is a deprecated alias for ``np.object0` is a deprecated alias for `np.object_`. `object` can be used instead.  (Deprecated NumPy 1.24)`.  (Deprecated NumPy 1.24)
  Object0 = np.object0
/opt/miniconda3/envs/hnx/lib/python3.11/site-packages/nptyping/typing_.py:66: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  Int0 = np.int0
/opt/miniconda3/envs/hnx/lib/python3.11/site-packages/nptyping/typing_.py:80: DeprecationWarning: `np.uint0` is a deprecated alias for `np.uintp`.  (Deprecated NumPy 1.24)
  UInt0 = np.uint0
/opt/miniconda3/envs/hnx/lib/python3.11/site-packages/nptyping/typing_.py:107: DeprecationWarning: `np.void0` is a deprecated alias for

## Maximum Hyperedge Triplets

In hypergraphs, hyperedge triplets are sequence-independent sets of three unique hyperedges. Hyperedge triplets vary in their connectivity patterns. In the following figure, we show the venn diagram of three hyperedges where each region represents the intersection between its corresponding hyperedges. We denote the three green regions as the *independent* regions, the three blue regions as the *disjoint* regions, and the red region as the *common* region.

<img src="images/ShadedTriplet.png" width="100" align="left" style="margin-right:10px">

Maximum hyperedge triplets are based on their independent, disjoint, and common weights.
These weights correspond to the three hyperedges which 
(1) are the least correlated with one another, 
(2) have the highest pairwise but not groupwise correlation, and 
(3) are the most correlated with one another, respectively.
We find maximum hyperedge triplets by iterating through hyperedges which can exceed the current maximum weight.

For a detailed explanation of maximum hyperedge triplets and the algorithms see: 
*Size-Aware Hyperedge Motifs*. 
When available, this will be replaced by the paper's citation.

In this tutorial, we will introduce how to run the main algorithms featured in the paper.

## Example on LesMis

We will run the maximum hyperedge triplet algorithms on the LesMis dataset. We construct a hypergraph $H$ with characters as nodes and books as hyperedges.

In [3]:
lm = hnx.LesMis()

### List of characters as they appear in each scene.
scenes = lm.df_scenes

### Construct a hypergraph
### The hyperedges will be the books and the nodes the characters.
fantine = scenes.loc[scenes.Volume == 1]
cosette = scenes.loc[scenes.Volume == 2]
marius = scenes.loc[scenes.Volume == 3]
stdenis = scenes.loc[scenes.Volume == 4]
jeanvaljean = scenes.loc[scenes.Volume == 5]

vols = [fantine, cosette, marius, stdenis, jeanvaljean]
names = ["fantine", "cosette", "marius", "stdenis", "jeanvaljean"]

book_edges = dict()
for idx in range(0,5):
    vol = vols[idx]
    name = names[idx]
    for book in vols[idx].Book:
        label = "Book #{book} in {name}".format(name=name, book=book)
        book_edges[label] = set(vols[idx].loc[vols[idx].Book == book]['Characters'])

H = hnx.Hypergraph(book_edges)

/Users/prag717/Library/CloudStorage/OneDrive-PNNL/Documents/tdm/hnx/hypernetx/utils/toys/lesmis.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.book_tour_data = self.df_scenes.groupby(["Volume", "Book"]).apply(


## Independent Weight

The independent weight of a hyperedge triplet is the minimum size of its independent regions divided by the sum of its disjoint and common regions with one.

In [4]:
max_triplet = hnxmt.max_independent(H)

print(max_triplet)

{'weight': 10.0, 'v1': 'Book #12 in stdenis', 'v2': 'Book #7 in fantine', 'v3': 'Book #1 in fantine'}


## Disjoint Weight

The disjoint weight of a hyperedge triplet is the minimum size of its disjoint regions divided by the sum of its common region and one.

In [5]:
max_triplet = hnxmt.max_disjoint(H)

print(max_triplet)

{'weight': 3.0, 'v1': 'Book #1 in jeanvaljean', 'v2': 'Book #8 in stdenis', 'v3': 'Book #6 in stdenis'}


## Common Weight

The common weight of a hyperedge triplet is the size of its common region.

In [6]:
max_triplet = hnxmt.max_common(H)

print(max_triplet)

{'weight': 8, 'v1': 'Book #1 in jeanvaljean', 'v2': 'Book #12 in stdenis', 'v3': 'Book #14 in stdenis'}


## Optional Speedup

For each algorithm, we include an optional parameter *min_weight* which only considers hyperedge triplets which have a weight of at least *min_weight*. Its default value is 0. The closer *min_weight* is to the maximum weight, the faster the algorithm runs.